In [2]:
%load_ext autoreload
%autoreload 2

# データ収集用Jupyterノートブック


In [1]:
# データ収集用Jupyterノートブック

import sys
import os
import json
import datetime as dt
import pandas as pd
import asyncio

# データコレクターモジュールをインポート
from data_collector import BTCDataCollector

# 設定ファイルから読み込む
def load_config(config_path="config/data_config.json"):
    try:
        with open(config_path, "r") as f:
            config = json.load(f)

        # 日付文字列をdatetimeオブジェクトに変換
        if "start_date" in config and config["start_date"]:
            config["start_date"] = dt.datetime.fromisoformat(config["start_date"])
        if "end_date" in config and config["end_date"]:
            config["end_date"] = dt.datetime.fromisoformat(config["end_date"])
        else:
            config["end_date"] = dt.datetime.now()

        return config
    except Exception as e:
        print(f"設定ファイルの読み込みエラー: {e}")
        return None

# 設定ファイルから読み込み
config = load_config()

# または直接設定
if config is None:
    config = {
        "exchange": "bybit",
        "symbol": "BTCUSDT",
        "timeframe": "5",
        "start_date": dt.datetime(2023, 1, 1),
        "end_date": dt.datetime.now(),
        "output_dir": "data/raw",
        "output_filename": "btcusd_5m_data.csv",
        "api_keys": {
            "bybit": ["YOUR_API_KEY", "YOUR_API_SECRET"]  # 実際のキーに置き換え
        },
        "use_direct_api": True  # True: 直接APIを使用, False: pybottersを使用
    }

# API キーを設定（必要な場合）
# config["api_keys"]["bybit"] = ["YOUR_ACTUAL_API_KEY", "YOUR_ACTUAL_API_SECRET"]

# 設定内容を表示（APIキーは隠す）
print_config = config.copy()
print_config["api_keys"] = {k: ["***", "***"] for k, v in config["api_keys"].items()}
print("使用する設定:")
for key, value in print_config.items():
    print(f"  {key}: {value}")

# データ収集と保存の関数
async def collect_and_save_data(config):
    """データを収集して保存する関数"""
    collector = BTCDataCollector(config)
    df = await collector.collect_historical_data()
    summary = collector.save_data(df)

    # サマリー出力
    print("\n=== データ収集結果サマリー ===")
    print(f"状態: {summary['status']}")
    print(f"行数: {summary['rows']}")
    print(f"期間: {summary['start_date']} から {summary['end_date']}")
    if summary['status'] == 'success':
        print(f"価格統計（終値）: 最小={summary['data_stats']['close']['min']:.2f}, "
              f"最大={summary['data_stats']['close']['max']:.2f}, "
              f"平均={summary['data_stats']['close']['mean']:.2f}")
        print(f"出来高合計: {summary['data_stats']['volume']['total']:.2f}")
        print(f"欠損値: {sum(summary['missing_values'].values())}")

    return df, summary

# データ収集実行
df, summary = await collect_and_save_data(config)

# データ確認
if 'df' in locals() and not df.empty:
    print("データサンプル（先頭5行）:")
    print(df.head())

    print("\n基本統計:")
    print(df.describe())

    print(f"\n合計レコード数: {len(df)}")
else:
    print("データの収集に失敗したか、データが空です。")

2025-04-26 01:20:11,089 - bybit_collector - INFO - BTCUSDTの5分足データを2023-01-01 00:00:00から2025-04-26 01:20:11.088789まで収集します
2025-04-26 01:20:11,090 - bybit_collector - INFO - データ取得を847個のチャンクに分割しました


使用する設定:
  exchange: bybit
  symbol: BTCUSDT
  timeframe: 5
  start_date: 2023-01-01 00:00:00
  end_date: 2025-04-26 01:20:11.088789
  output_dir: data/raw
  output_filename: btcusd_5m_data.csv
  api_keys: {'bybit': ['***', '***']}
  use_direct_api: True


データ取得:   0%|          | 0/847 [00:00<?, ?it/s]

2025-04-26 01:20:26,578 - bybit_collector - INFO - データ収集完了。合計 169216 行のデータを取得
2025-04-26 01:20:27,195 - bybit_collector - INFO - データを data\raw\btcusd_5m_data.csv に保存しました。データサイズ: (169216, 6)



=== データ収集結果サマリー ===
状態: success
行数: 169216
期間: 2022-12-31T22:25:00 から 2025-04-25T16:20:00
価格統計（終値）: 最小=16508.00, 最大=109059.40, 平均=53358.38
出来高合計: 65739890.39
欠損値: 0
データサンプル（先頭5行）:
                        open     high      low    close   volume      turnover
timestamp                                                                     
2022-12-31 22:25:00  16545.0  16568.5  16544.5  16557.0  199.789  3.308472e+06
2022-12-31 22:30:00  16557.0  16569.0  16557.0  16563.0  265.757  4.402056e+06
2022-12-31 22:35:00  16563.0  16563.0  16544.5  16546.0  165.466  2.738463e+06
2022-12-31 22:40:00  16546.0  16552.0  16528.5  16551.5  295.414  4.885751e+06
2022-12-31 22:45:00  16551.5  16552.0  16533.0  16534.0   78.281  1.295245e+06

基本統計:
                open           high            low          close  \
count  169216.000000  169216.000000  169216.000000  169216.000000   
mean    53358.105046   53406.209525   53309.329747   53358.383285   
std     25347.259013   25375.180359   25319.261788  

# 特徴量エンジニアリング用Jupyter Notebook


In [1]:
# 特徴量エンジニアリング用Jupyter Notebook

import sys
import os
import json
import pandas as pd
import numpy as np
from IPython.display import display

# data_processorモジュールをインポート
sys.path.append(os.path.abspath(os.getcwd()))
from data_processor import generate_features

# 特徴量生成の設定（必要に応じてカスタマイズ）
config = {
    "input_dir": "data/raw",
    "input_filename": "btcusd_5m_data.csv",
    "output_dir": "data/processed",
    "output_filename": "btcusd_5m_features.csv",
    "features": {
        "price_change": True,
        "volume_change": True,
        "moving_averages": True,
        "rsi": True,
        "high_low_distance": True,
        "bollinger_bands": True,
        "macd": True,
        "stochastic": True
    },
    "ma_periods": [5, 10, 20, 50, 100, 200],
    "rsi_period": 14,
    "bollinger_period": 20,
    "bollinger_std": 2,
    "macd_params": {"fast": 12, "slow": 26, "signal": 9},
    "stochastic_params": {"k": 14, "d": 3, "slowing": 3},
    "target_periods": [1, 2, 3],
    "classification_threshold": 0.0005
}

# 特徴量生成の実行
feature_df, report = generate_features(config)

# 結果の表示
print("\n=== 特徴量生成結果 ===")
print(f"状態: {report['status']}")
print(f"行数: {report['rows']}")
print(f"列数: {report['columns']}")
print(f"期間: {report['start_date']} から {report['end_date']}")
print(f"欠損値: {report['missing_values']}")

print("\n特徴量グループごとの数:")
for group, count in report['feature_counts'].items():
    print(f"  {group}: {count}")

print("\n目標変数の分布:")
for target, distribution in report['target_distribution'].items():
    print(f"  {target}: {distribution}")

# 主要特徴量の基本統計量を表示
main_features = [
    'close', 'volume',
    'price_change_pct_1', 'volume_change_pct_1',
    'rsi', 'bb_width', 'macd', 'stoch_k',
    'target_price_change_pct_1'
]

if all(feat in feature_df.columns for feat in main_features):
    print("\n主要特徴量の基本統計量:")
    stats = feature_df[main_features].describe()
    for col in stats.columns:
        print(f"\n{col}:")
        for stat, value in stats[col].items():
            print(f"  {stat}: {value:.6f}")

# 相関係数の表示（主要な特徴量のみ）
if all(feat in feature_df.columns for feat in main_features):
    main_features_with_target = main_features + ['target_price_direction_1']
    print("\n主要特徴量と目標変数(1期先)の相関係数:")
    target_corr = feature_df[main_features_with_target].corr()['target_price_direction_1'].drop('target_price_direction_1')
    for feat, corr in target_corr.items():
        print(f"  {feat}: {corr:.6f}")

# 特徴量の頻度カウント（カテゴリ変数）
for period in config["target_periods"]:
    direction_col = f'target_price_direction_{period}'
    if direction_col in feature_df.columns:
        value_counts = feature_df[direction_col].value_counts()
        total = value_counts.sum()
        print(f"\n{direction_col} の頻度分布:")
        for value, count in value_counts.items():
            percentage = (count / total) * 100
            print(f"  {value}: {count} ({percentage:.2f}%)")

# 特徴量間の相関行列（数値のみ表示）
if 'target_price_direction_1' in feature_df.columns:
    print("\n特徴量と目標変数の相関（絶対値が0.05以上のもののみ）:")
    corrs = feature_df.corr()['target_price_direction_1'].drop('target_price_direction_1')
    significant_corrs = corrs[abs(corrs) >= 0.05].sort_values(ascending=False)
    for feat, corr in significant_corrs.items():
        print(f"  {feat}: {corr:.6f}")

# データサンプル数値表示（最初の5行の主要特徴量のみ）
print("\n最初の5行のデータサンプル（主要特徴量のみ）:")
sampled_features = ['close', 'volume', 'rsi', 'macd', 'target_price_change_pct_1', 'target_price_direction_1']
if all(feat in feature_df.columns for feat in sampled_features):
    sample_df = feature_df[sampled_features].head(5)
    for i, row in sample_df.iterrows():
        print(f"\n行 {i}:")
        for col, val in row.items():
            print(f"  {col}: {val:.6f}" if isinstance(val, float) else f"  {col}: {val}")

2025-04-26 01:33:23,065 - feature_engineering - INFO - データを data\raw\btcusd_5m_data.csv から読み込みます
2025-04-26 01:33:23,215 - feature_engineering - INFO - 169216 行のデータを読み込みました
2025-04-26 01:33:23,216 - feature_engineering - INFO - 価格変動率の特徴量を追加しています
2025-04-26 01:33:23,279 - feature_engineering - INFO - 出来高変動率の特徴量を追加しています
2025-04-26 01:33:23,314 - feature_engineering - INFO - 移動平均線と乖離率の特徴量を追加しています
2025-04-26 01:33:23,350 - feature_engineering - INFO - RSIの特徴量を追加しています
2025-04-26 01:33:23,365 - feature_engineering - INFO - 高値/安値からの距離の特徴量を追加しています
2025-04-26 01:33:23,411 - feature_engineering - INFO - ボリンジャーバンドの特徴量を追加しています
2025-04-26 01:33:23,424 - feature_engineering - INFO - MACDの特徴量を追加しています
2025-04-26 01:33:23,435 - feature_engineering - INFO - ストキャスティクスの特徴量を追加しています
2025-04-26 01:33:23,457 - feature_engineering - INFO - 予測対象の目標変数を追加しています
2025-04-26 01:33:23,569 - feature_engineering - INFO - 特徴量を生成しました。カラム数: 111, 行数: 169014
2025-04-26 01:33:35,839 - feature_engineering - INFO - 特徴量を data\pr


=== 特徴量生成結果 ===
状態: success
行数: 169014
列数: 111
期間: 2023-01-01T15:00:00 から 2025-04-25T16:05:00
欠損値: 0

特徴量グループごとの数:
  price_change: 20
  volume_change: 17
  moving_averages: 20
  rsi: 5
  high_low_distance: 8
  bollinger_bands: 8
  macd: 4
  stochastic: 5
  target: 6

目標変数の分布:
  target_price_direction_1: {0: 74629, 1: 47342, -1: 47043}
  target_price_direction_2: {1: 56458, 0: 56410, -1: 56146}
  target_price_direction_3: {1: 60957, -1: 60464, 0: 47593}

主要特徴量の基本統計量:

close:
  count: 169014.000000
  mean: 53400.987552
  std: 25330.507231
  min: 16560.500000
  25%: 28497.250000
  50%: 54177.950000
  75%: 68722.150000
  max: 109059.400000

volume:
  count: 169014.000000
  mean: 388.843145
  std: 760.230836
  min: 1.612000
  25%: 104.731000
  50%: 197.637500
  75%: 396.670000
  max: 43836.312000

price_change_pct_1:
  count: 169014.000000
  mean: 0.000012
  std: 0.001888
  min: -0.090764
  25%: -0.000583
  50%: 0.000000
  75%: 0.000590
  max: 0.107624

volume_change_pct_1:
  count: 169014